In [1]:
USERNAME = 'jnahom'
import pandas as pd
from datetime import datetime, timedelta
import forecast_functions

In [14]:
#Data
last_ten_alb = pd.read_csv('last_ten_alb10_03_2021.csv')
last_ten_alb

,Unnamed: 0,STNID,NAME,CTRY,COUNTRY_NAME,ISO2C,ISO3C,STATE,LATITUDE,LONGITUDE,...,I_FOG,I_RAIN_DRIZZLE,I_SNOW_ICE,I_HAIL,I_THUNDER,I_TORNADO_FUNNEL,EA,ES,RH,I_PRCP
0,1,725180-14735,ALBANY INTERNATIONAL AIRPORT,US,UNITED STATES,US,USA,NY,42.747,-73.799,...,0,0,0.0,0.0,0.0,0.0,0.6,0.9,67.1,0
1,2,725180-14735,ALBANY INTERNATIONAL AIRPORT,US,UNITED STATES,US,USA,NY,42.747,-73.799,...,1,0,0.0,0.0,0.0,NaN,0.7,0.9,75.4,1
2,3,725180-14735,ALBANY INTERNATIONAL AIRPORT,US,UNITED STATES,US,USA,NY,42.747,-73.799,...,1,0,0.0,0.0,NaN,NaN,0.3,0.5,59.7,1
3,4,725180-14735,ALBANY INTERNATIONAL AIRPORT,US,UNITED STATES,US,USA,NY,42.747,-73.799,...,1,0,0.0,0.0,NaN,NaN,0.3,0.5,67.7,0
4,5,725180-14735,ALBANY INTERNATIONAL AIRPORT,US,UNITED STATES,US,USA,NY,42.747,-73.799,...,0,0,0.0,0.0,0.0,0.0,0.3,0.5,68.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3924,3925,725180-14735,ALBANY INTERNATIONAL AIRPORT,US,UNITED STATES,US,USA,NY,42.747,-73.799,...,0,0,0.0,0.0,0.0,0.0,1.0,1.4,73.4,0
3925,3926,725180-14735,ALBANY INTERNATIONAL AIRPORT,US,UNITED STATES,US,USA,NY,42.747,-73.799,...,1,0,0.0,0.0,0.0,NaN,1.2,1.4,84.7,0
3926,3927,725180-14735,ALBANY INTERNATIONAL AIRPORT,US,UNITED STATES,US,USA,NY,42.747,-73.799,...,1,0,0.0,0.0,0.0,0.0,1.0,1.3,72.8,0
3927,3928,725180-14735,ALBANY INTERNATIONAL AIRPORT,US,UNITED STATES,US,USA,NY,42.747,-73.799,...,0,0,0.0,0.0,0.0,0.0,1.2,1.6,74.3,0


In [15]:
# set YEARMODA (Year-month-day) to datetime format
last_ten_alb['YEARMODA'] = pd.to_datetime(last_ten_alb['YEARMODA'])
print("Most recent date in data:", last_ten_alb['YEARMODA'].max())

Most recent date in data: 2021-10-03 00:00:00


In [4]:
# get date from user to start forecasting
prompt_str = "When should the prediction start? Enter date in YYYY-MM-DD [Enter=tomorrow]: "
prediction_start = input(prompt_str)

When should the prediction start? Enter date in YYYY-MM-DD [Enter=tomorrow]: 2021-10-06


In [5]:
# user hits enter, then just give tomorrow's date
if not prediction_start:
    prediction_start = datetime.today() + timedelta(days=1)
    prediction_start = prediction_start.strftime("%Y-%m-%d")

In [6]:
# establish important dates
prediction_start = datetime.strptime(prediction_start, "%Y-%m-%d")
start_prev_seven = prediction_start - timedelta(days=7)

In [7]:
# create list of dates to submit for forecast evaluation
eval_dates = pd.date_range(prediction_start, periods=7).tolist()

In [16]:
#%% create training subset
train_dates = []
for i in range(0,12):
    #print(i)
    train_dates_temp = pd.date_range(start_prev_seven - timedelta(days=365*i), periods=7)
    train_dates_temp = train_dates_temp.tolist()
    for j in train_dates_temp:
        train_dates.append(j)

#print(train_dates)
#train_dates = pd.DatetimeIndex(train_dates.values)
print(train_dates)
train_data = last_ten_alb.loc[last_ten_alb['YEARMODA'].isin(train_dates)]

[Timestamp('2021-09-29 00:00:00', freq='D'), Timestamp('2021-09-30 00:00:00', freq='D'), Timestamp('2021-10-01 00:00:00', freq='D'), Timestamp('2021-10-02 00:00:00', freq='D'), Timestamp('2021-10-03 00:00:00', freq='D'), Timestamp('2021-10-04 00:00:00', freq='D'), Timestamp('2021-10-05 00:00:00', freq='D'), Timestamp('2020-09-29 00:00:00', freq='D'), Timestamp('2020-09-30 00:00:00', freq='D'), Timestamp('2020-10-01 00:00:00', freq='D'), Timestamp('2020-10-02 00:00:00', freq='D'), Timestamp('2020-10-03 00:00:00', freq='D'), Timestamp('2020-10-04 00:00:00', freq='D'), Timestamp('2020-10-05 00:00:00', freq='D'), Timestamp('2019-09-30 00:00:00', freq='D'), Timestamp('2019-10-01 00:00:00', freq='D'), Timestamp('2019-10-02 00:00:00', freq='D'), Timestamp('2019-10-03 00:00:00', freq='D'), Timestamp('2019-10-04 00:00:00', freq='D'), Timestamp('2019-10-05 00:00:00', freq='D'), Timestamp('2019-10-06 00:00:00', freq='D'), Timestamp('2018-09-30 00:00:00', freq='D'), Timestamp('2018-10-01 00:00:00'

In [18]:
#%% get average values based on last 7 days
avg_max = train_data['MAX'].mean()
avg_min = train_data['MIN'].mean()
avg_prcp = train_data['I_PRCP'].mean()

0.4533333333333333

In [25]:
#%% create training subset
max_dict = {}
min_dict = {}
prcp_dict = {}

for date in eval_dates:
    train_dates_list = []
    for i in range(0,12):
        #print(i)
        train_dates_temp = date - timedelta(days=365*i)
        train_dates_list.append(train_dates_temp)
    #print(train_dates_temp)
    #print(train_dates)
    #train_dates = pd.DatetimeIndex(train_dates.values)
    #print(train_dates_temp)
    train_data_loop = last_ten_alb.loc[last_ten_alb['YEARMODA'].isin(train_dates_list)]
    #print(train_data_loop)
    avg_max_loop = train_data_loop['MAX'].mean()
    avg_min_loop = train_data_loop['MIN'].mean()
    avg_prcp_loop = train_data_loop['I_PRCP'].mean()
    
    max_dict[date] = avg_max_loop
    min_dict[date] = avg_min_loop
    prcp_dict[date] = avg_prcp_loop
    #print(avg_prcp_loop)
    #print(train_data_loop['MIN'])
    
print(max_dict)
print(min_dict)
print(prcp_dict)

{Timestamp('2021-10-06 00:00:00', freq='D'): 21.44, Timestamp('2021-10-07 00:00:00', freq='D'): 21.21, Timestamp('2021-10-08 00:00:00', freq='D'): 20.71, Timestamp('2021-10-09 00:00:00', freq='D'): 19.94, Timestamp('2021-10-10 00:00:00', freq='D'): 21.15, Timestamp('2021-10-11 00:00:00', freq='D'): 20.310000000000002, Timestamp('2021-10-12 00:00:00', freq='D'): 19.330000000000002}
{Timestamp('2021-10-06 00:00:00', freq='D'): 8.22, Timestamp('2021-10-07 00:00:00', freq='D'): 7.49, Timestamp('2021-10-08 00:00:00', freq='D'): 7.49, Timestamp('2021-10-09 00:00:00', freq='D'): 7.3199999999999985, Timestamp('2021-10-10 00:00:00', freq='D'): 6.26, Timestamp('2021-10-11 00:00:00', freq='D'): 5.720000000000001, Timestamp('2021-10-12 00:00:00', freq='D'): 7.799999999999999}
{Timestamp('2021-10-06 00:00:00', freq='D'): 0.6, Timestamp('2021-10-07 00:00:00', freq='D'): 0.3, Timestamp('2021-10-08 00:00:00', freq='D'): 0.4, Timestamp('2021-10-09 00:00:00', freq='D'): 0.2, Timestamp('2021-10-10 00:00:

In [27]:
def construct_mean_forecast_df(fc_dates, username, fc_var, avg_val, avg_dict={}):
    """
    Create a dictionary to add to the forecast dataframe.
    This version repeats the average value of a variable
    over the number of days.
    
    This function could be copied and modified to accept
    a list of forecast values instead of a single value
    repeated.
    Parameters
    ----------
    fc_dates : list 
        list of dates in the forecast
    avg_val : float
        Value to repeat across days.
    fc_var : str
        Name of value you are forecasting. Should be 
        one of MAX, MIN, P_PRCP.
    Returns
    -------
    mean_df : pandas.DataFrame()
        Dataframe with column names and list of values for 
        each column.
    """
    
    nrow = len(fc_dates)
    fc_value = []
    
    for i in fc_dates:
        print(i)
        print((avg_val+avg_dict[i])/2)
        fc_value.append((avg_val+avg_dict[i])/2)
        
    mean_dict = {'fc_date': fc_dates, 
              'fc_name': [username] * nrow, 
              'fc_var': [fc_var] * nrow, 
              'fc_value': fc_value}
    #print(mean_dict)
    mean_df = pd.DataFrame.from_dict(mean_dict)
    return mean_df

In [28]:
# format forecasts
fc_max = construct_mean_forecast_df(eval_dates, USERNAME, 'MAX', avg_max, max_dict)
fc_min = construct_mean_forecast_df(eval_dates, USERNAME, 'MIN', avg_min, min_dict)
fc_prcp = construct_mean_forecast_df(eval_dates, USERNAME, 'P_PRCP', avg_prcp, prcp_dict)
fc_prcp

2021-10-06 00:00:00
20.756
2021-10-07 00:00:00
20.641000000000002
2021-10-08 00:00:00
20.391000000000002
2021-10-09 00:00:00
20.006
2021-10-10 00:00:00
20.611
2021-10-11 00:00:00
20.191000000000003
2021-10-12 00:00:00
19.701
2021-10-06 00:00:00
8.144000000000002
2021-10-07 00:00:00
7.779000000000001
2021-10-08 00:00:00
7.779000000000001
2021-10-09 00:00:00
7.694
2021-10-10 00:00:00
7.164000000000001
2021-10-11 00:00:00
6.894000000000001
2021-10-12 00:00:00
7.934
2021-10-06 00:00:00
0.5266666666666666
2021-10-07 00:00:00
0.37666666666666665
2021-10-08 00:00:00
0.42666666666666664
2021-10-09 00:00:00
0.32666666666666666
2021-10-10 00:00:00
0.37666666666666665
2021-10-11 00:00:00
0.4766666666666667
2021-10-12 00:00:00
0.4766666666666667


,fc_date,fc_name,fc_var,fc_value
0,2021-10-06,jnahom,P_PRCP,0.526667
1,2021-10-07,jnahom,P_PRCP,0.376667
2,2021-10-08,jnahom,P_PRCP,0.426667
3,2021-10-09,jnahom,P_PRCP,0.326667
4,2021-10-10,jnahom,P_PRCP,0.376667
5,2021-10-11,jnahom,P_PRCP,0.476667
6,2021-10-12,jnahom,P_PRCP,0.476667


In [29]:
# define columns
column_names = ['fc_date', 'fc_name', 'fc_var', 'fc_value']
# create empty df
eval_df_base = pd.DataFrame(columns=column_names)

In [30]:
# append forecasts to dataframe
eval_df_base = eval_df_base.append(fc_max, ignore_index=True)
eval_df_base = eval_df_base.append(fc_min, ignore_index=True)
eval_df_base = eval_df_base.append(fc_prcp, ignore_index=True)

In [31]:
#%% save output
outfile_name = USERNAME + '_' + prediction_start.strftime("%Y-%m-%d") + '.csv'
eval_df_base.to_csv(outfile_name, index=False)